In [ ]:
import numpy as np
import sep
import glob as glob

In [ ]:
import astropy.io
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

# Find UDF f105w file and create image

Locate UDF_f105w image data (Kyle Davis)

In [ ]:
filepool = glob.glob("*f105w" + "*drz.fits")
file = filepool[0]
print(file)

read image into standard 2-d array (Kyle Davis)

In [ ]:
image = fits.open(file)
data = image[0].data

Change byte order of array to a native byte order (Kyle Davis)

In [ ]:
data = data.byteswap(inplace=True).newbyteorder()
print(data)

Show the image using mean and standard deviation to assign the colormap (Miles Pulk)

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('UDF_f105w_raw.png')

# Background subtraction (All Miles Pulk)

 measure a spatially varying background on the image

In [ ]:
bkg = sep.Background(data)#, mask=mask, bw=64, bh=64, fw=3, fh=3)

Determine the 'global' mean and noise of the background created

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

Convert the background to be a 2-d array -- the same size as the original image

In [ ]:
bkg_image = bkg.back()

Show the background

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('UDF_f105w_bkg.png')

evaluate the background noise as 2-d array, same size as original image


In [ ]:
bkg_rms = bkg.rms()

show the background noise


In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('UDF_f105w_bkgnoise.png')

subtract the background from the image

In [ ]:
data_sub = data - bkg

Show the image without background

In [ ]:
m, s = np.mean(data_sub), np.std(data_sub)
plt.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

# Object Detection (All Miles Pulk)

Run object detection with the detection threshold to be 1.5 times the global background RMS

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

Display number of detected objects

In [ ]:
len(objects)

Plot the new background-subtracted image and a red ellipse around each object detected

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('UDF_f105w_sources.png')

Perform simple circular aperture photometry with 3 pixel radii at the object locations and gather data

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

Display flux for each object

In [ ]:
# show the results:
for i in range(8645):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
print(flux)

# Convert Flux to AB Magnitude (All Kyle Davis)

Remove flux values that are detected negative due to precision limitations

In [ ]:
flux = flux[flux >= 0]
print(flux)

Define function to find reference magnitude using STSCI conversion and .fits header data

In [ ]:
def find_ZP_AB(PHOTFLAM, PHOTPLAM):
    ZP_AB = -2.5*np.log10(PHOTFLAM) - 5*np.log10(PHOTPLAM) - 2.408
    return ZP_AB

Define function to convert flux to AB Mag

In [ ]:
def AB_Mag_convert(flux, Mag_ref):
    AB_Mag = -2.5*np.log10(flux) + Mag_ref
    return AB_Mag

Find reference magnitude

In [ ]:
ZP = find_ZP_AB( 3.0385782e-20, 1.0552033e04)
print(ZP)

Convert flux to AB Magnitude

In [ ]:
AB_Mag = AB_Mag_convert(flux, ZP)

print(AB_Mag, len(AB_Mag))

Histogram the fluxes

In [ ]:
plt.figure()
plt.xlabel('AB Magnitude')
plt.ylabel('Frequency')
plt.hist(AB_Mag, 20)